Les importations de modules :

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import scipy
import numpy as np
from scipy.special import kv, iv
import os


Les fonctions utilisées sont :

ouvrir un fichier et return une liste avec les valeurs (A utiliser pour faire les graphiques)

In [4]:

def ouverture_fichier(nomfichier):
    with open(nomfichier, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        xc = []
        i = 0
        for row in csvreader:
            xc.append(float(row[1]))
            i += 1
            if i >= 1000:
                break
    return xc

Parametres du circuit

In [ ]:
# Paramètres du circuit
omega = 0                       # Fréquence angulaire en rad/s
mu_0 = 0                        # Perméabilité du vide en Tm/A
mu = 0                          # Permeabilite du materiaux
mu_m = mu / mu_0                # µ/µ_0
a = 0                           # Rayon du solénoïde en m
n = 0                           # Nombre de spires du solénoïde
epsilon_0 = 0                   # Permittivité du vide en F/m
epsilon = 0                     # Permittivité de la tige en F/m
b = 0                           # Rayon de la tige conductrice en m
rho = 0                         # Résistivité de la tige conductrice en ohm/m

# Calculs supplémentaires
zeta = np.sqrt((omega ** 2) * mu_0 * epsilon_0)
eta = np.sqrt(zeta ** 2 - (omega ** 2) * mu_0 * epsilon_0)
gamma = b * np.sqrt(zeta ** 2 + (1j * omega * mu_m / rho) - (omega ** 2) * mu_m * epsilon)

# Fonctions de Bessels modifiées
I_0 = iv(0, gamma)
I_1 = iv(1, gamma)
K_0 = kv(0, eta * a)
K_1 = kv(1, eta * a)

Production CSV pour resistivite (finir la premiere fonciton : "calcul_resistivite")
Le reste devrait fonctioner

In [3]:
def calcul_resitivite() :
    Liste = []
    x = [1,2,3]
    for i in x :
        Liste.append(i)

    return Liste


def list_to_csv(input_list, file_name):

    # Obtenir le chemin du dossier actuel
    current_directory = os.getcwd()
    # Combiner le chemin du dossier avec le nom du fichier
    file_path = os.path.join(current_directory, file_name)
    
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in input_list:
            writer.writerow([row])


def Calcul_total(file_name) :
    Lresultat = calcul_resitivite()
    list_to_csv(Lresultat, file_name)

file = "Resistivite"
Calcul_total(file)
    

Generation de Graphiques :

In [ ]:

#Recevoir/ouvrir CSV thermometre et resitivite

nom_fichier_temperature = "..."     # Remplacez "mon_fichier.csv" par le nom de votre fichier CSV
                                
nom_fichier_resistivite = "..."     # Remplacez "mon_fichier.csv" par le nom de votre fichier CSV

#Faire les variables x = resistivite | y = temperature

x = ouverture_fichier(nom_fichier_temperature)
print('temperature',x)  

y = ouverture_fichier(nom_fichier_resistivite)
print('resistivite',y)  

# Tracer le graphique
plt.plot(x, y)  
plt.xlabel('Température')  
plt.ylabel('Résistivité')  
plt.title('Graphique de la résistivite de l\'echantillon en fonction de la temperature')  
plt.grid(True)  
plt.show()  


Calcul des incertitudes

In [ ]:
#code
